In [ ]:
using Gridap
using Gridap.Fields
using Gridap.Geometry
using GridapGmsh

using ForwardDiff

# Geometry

In [ ]:
# Enclosure dimensions (enclosure is not centered, the core is)
hencl1 = 65.5e-2;   # Height above the x-axis
hencl2 = -53.5e-2;  # Height below the x-axis
wencl  = 104e-2;    # Width

# Core dimensions
wcore = 84e-2;
hcore = 100e-2;

# Core gap dimensions (left and right are identical)
wgap = 17e-2;
hgap = 76e-2;
mgap = 17e-2;

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Mesh densities
lc1 = 2e-2;      # Enclosure & core outer
lc2 = 1e-2;      # Core inner
lc3 = 1e-2;      # HV windings
lc4 = 1e-2;      # LV windings

# Gridap Setup

In [ ]:
# Define norm of a complex vector for Gridap to use on the CellField which results from Bh = ∇(uh)
norm_complex(a::VectorValue{2, ComplexF64}) = sqrt(norm(a[1])^2 + norm(a[2])^2)
norm_complex(a::CellField) = Operation(norm_complex)(a)

# Linear BH Curve

In [ ]:
# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin.msh")
Ω = Triangulation(model)

# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)
dΩ = Measure(Ω, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
V = TestFESpace(model, reffe, conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
U = TrialFESpace(V, [0])

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

mu0 = 4e-7 * pi;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

In [ ]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Ω)

const tag_core = get_tag_from_name(labels, "Core");

const tag_lv1l = 10; const tag_lv1r = 11;
const tag_lv2l = 12; const tag_lv2r = 13;
const tag_lv3l = 14; const tag_lv3r = 15;

# Source current density
function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Js * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return -Js * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Js;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return -Js;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Js * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return -Js * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

# Permeability model
function fmur(tag)
    if tag == tag_core
        return 1000.0;
    else
        return 1.0;
    end
end

function fnu(tag)
    return 1 / (mu0 * fmur(tag));
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 0.1;
    else
        return 0.0;
    end
end

In [ ]:
# Define the weak form (bilinear and linear terms)
a(u,v) = ∫( (fnu ∘ τ) ⋅ ∇(u) ⋅ ∇(v) )dΩ + ∫( 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ u ⋅ v )dΩ
b(v)   = ∫( (fsource ∘ τ) ⋅ v )dΩ;
op = AffineFEOperator(a, b, U, V)

# Solve the linear FE system with LU solver
ls = LUSolver()
solver = LinearFESolver(ls)
uh = solve(solver, op);

In [ ]:
# Post-processing for magnetic field and current density
Bh = norm_complex(gradient(uh));

J0    = (fsource ∘ τ); # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [ ]:
writevtk(Ω, "images/transformer_gridap/transformer1", cellfields=["Az"=>abs(uh), "imAz"=>imag(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy)])

![Result: Magnetic Flux Density](images/transformer_gridap/transformer1.png)

# Non-linear BH Curve

In [ ]:
# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin.msh")
#model = GmshDiscreteModel("geo/transformer_stedin_hybrid.msh")    # This works if GridapGmsh is modified to allow multiple element types per dimension
Ω = Triangulation(model)

# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)
dΩ = Measure(Ω, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
V = TestFESpace(model, reffe, conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
U = TrialFESpace(V, [0])

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

mu0 = 4e-7 * pi;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

In [ ]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Ω)

const tag_core = get_tag_from_name(labels, "Core");

const tag_lv1l = 10; const tag_lv1r = 11;
const tag_lv2l = 12; const tag_lv2r = 13;
const tag_lv3l = 14; const tag_lv3r = 15;

# Source current density
function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Js * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return -Js * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Js;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return -Js;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Js * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return -Js * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

# Permeability model
bh_a = 2.12e-4; 
bh_b = 7.358;
bh_c = 1.18e7;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));
fdmur_core(B) = ForwardDiff.derivative(fmur_core, B);

function fmur(tag, ∇u)
    if tag == tag_core
        normB = norm_complex(∇u);
        return fmur_core(normB);
    else
        return 1.0;
    end
end

function fdmur(tag, ∇du, ∇u)
    if tag == tag_core
        normB = norm_complex(∇u);
        return fdmur_core(normB) ⋅ (∇du ⊙ ∇u);
    else
        return 0.0 + 0.0im;
    end
end

function fnu(tag, ∇u)
    return 1 / (mu0 * fmur(tag, ∇u));
end

function fdnu(tag, ∇du, ∇u)
    return -1 / (mu0 * fmur(tag, ∇u)^2) * fdmur(tag, ∇du, ∇u);
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 1.0;
    else
        return 0.0;
    end
end

In [ ]:
# Define the weak form (bilinear and linear terms)
res(u,v) = ∫( (fnu ∘ (τ, ∇(u))) ⋅ ∇(u) ⋅ ∇(v) )dΩ + ∫( 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ u ⋅ v )dΩ - ∫( (fsource ∘ τ) ⋅ v )dΩ;
jac(u, du, v) = ∫( (fdnu ∘ (τ, ∇(du), ∇(u))) ⋅ ∇(u) ⋅ ∇(v) )dΩ + ∫( (fnu ∘ (τ, ∇(u))) ⋅ ∇(du) ⋅ ∇(v) )dΩ + ∫( 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ du ⋅ v )dΩ;;
op = FEOperator(res, jac, U, V)

ls = BackslashSolver()
nls = NLSolver(ls, method = :trust_region, show_trace = true)
solver = FESolver(nls)

x   = zeros(Complex{Float64}, num_free_dofs(U))
uh0 = FEFunction(U, x)
uh, = solve!(uh0, solver, op)

In [ ]:
# Post-processing for magnetic field and current density
Bh = norm_complex(∇(uh)); # Magnetic flux density
mu_r = fmur ∘ (τ, ∇(uh)); # Relative permeability

J0    = fsource ∘ τ; # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [ ]:
writevtk(Ω, "images/transformer_gridap/transformer2", cellfields=["Az"=>abs(uh), "imAz"=>imag(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy), "mur" => mu_r])

![Result: Magnetic Flux Density](images/transformer_gridap/transformer2.png)

![Result: Relative Permeability](images/transformer_gridap/transformer2_mu.png)

# Using Hybrid Meshes in Gridap
At the moment, ``GridapGmsh`` does not allow meshes with more than one element type per dimension (such as hybrid triangle-quad meshes). However, all of the functionality to allow (simple) hybrid meshes is present. I will try to submit a pull request or open an issue to extend this functionality. We only require changes to internal function ``_setup_reffes``.

In [ ]:
function _setup_reffes(gmsh,d,orient_if_simplex)
    elemTypes, elemTags, nodeTags = gmsh.model.mesh.getElements(d)

    ncells, nmin, nmax = _check_cell_tags(elemTags)
    cell_to_type = fill(Int8(0), ncells)
    for (i, etype) in enumerate(elemTypes)
        cell_to_type[elemTags[i] .- nmin .+ 1] .= i   # Correctly fill the cell_to_type array, instead of filling '1'
    end

    # TODO implement this check for hybrid meshes
    #etype::Int = first(elemTypes)
    #name, dim, order::Int, numv, parv = gmsh.model.mesh.getElementProperties(etype)

    #if order == 0 && etype == POINT
    #    order = 1
    #end

    #if order != 1
    #    gmsh.finalize()
    #    error("For the moment only for first-order elements")
    #end

    
    reffes = [_reffe_from_etype(etype) for etype in elemTypes];

    # How to implement this check for hybrid meshes... This could get tricky if some parts are oriented, and others are not?
    #boo = is_simplex(get_polytope(reffe)) && orient_if_simplex
    boo = false;

    orientation = boo ? Oriented() : NonOriented()

    (cell_to_type, reffes, orientation)
end

![Result: Magnetic Flux Density](images/transformer_gridap/transformer4.png)